# 第 4 单元：使用 PyTorch 编写你的第一个深度强化学习算法：Reinforce，并测试其鲁棒性 💪

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/thumbnail.png" alt="thumbnail"/>


在本 Notebook 中，你将从头开始编写你的第一个深度强化学习算法：Reinforce (也称为蒙特卡洛策略梯度)。

Reinforce 是一种*基于策略的方法*：一种**直接尝试优化策略而不使用动作价值函数**的深度强化学习算法。

更准确地说，Reinforce 是一种*策略梯度方法*，它是*基于策略的方法*的一个子类，旨在使用**梯度上升法估计最优策略的权重来直接优化策略**。

为了测试其鲁棒性，我们将在 2 个不同的简单环境中训练它：
- Cartpole-v1
- PixelcopterEnv

⬇️ 这是**你将在本 Notebook 结束时实现**的效果示例。⬇️

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/envs.gif" alt="Environments"/>


### 🎮 环境:

- [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)
- [PixelCopter](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)

### 📚 RL 库:

- Python
- PyTorch


我们一直在努力改进我们的教程，因此**如果你在本 Notebook 中发现任何问题**，请在[GitHub 仓库中提出问题](https://github.com/huggingface/deep-rl-class/issues)。

## 本 Notebook 的目标 🏆
在本 Notebook 结束时，你将能够：
- **使用 PyTorch 从零开始编写 Reinforce 算法。**
- **使用简单的环境测试你的智能体的鲁棒性。**
- **将你训练好的智能体推送到 Hub**，并附上精彩的回放视频和评估分数 🔥。

## 本 Notebook 来自深度强化学习课程
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

在这个免费课程中，你将：

- 📖 **在理论和实践中**学习深度强化学习。
- 🧑‍💻 学习**使用著名的深度 RL 库**，如 Stable Baselines3、RL Baselines3 Zoo、CleanRL 和 Sample Factory 2.0。
- 🤖 在**独特的环境中训练智能体**

更多内容请查看 📚 课程大纲 👉 https://simoninithomas.github.io/deep-rl-course

别忘了**<a href="http://eepurl.com/ic5ZUD">注册课程</a>** (我们收集你的电子邮件是为了**在每个单元发布时向你发送链接，并为你提供有关挑战和更新的信息**)。


保持联系的最佳方式是加入我们的 Discord 服务器，与社区和我们交流 👉🏻 https://discord.gg/ydHrjt3WP5

## 先决条件 🏗️
在深入学习本 Notebook 之前，你需要：

🔲 📚 [通过阅读第 4 单元来学习策略梯度](https://huggingface.co/deep-rl-course/unit4/introduction)

# 让我们从头开始编写 Reinforce 算法 🔥


要通过认证流程验证此实践项目，你需要将训练好的模型推送到 Hub。

- `Cartpole-v1` 的结果需 >= 350。
- `PixelCopter` 的结果需 >= 5。

要找到你的结果，请转到排行榜并找到你的模型，**结果 = mean_reward - std of reward**。**如果你在排行榜上看不到你的模型，请转到排行榜页面底部，然后单击刷新按钮**。

有关认证流程的更多信息，请查看此部分 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process


## 检查 GPU 支持 💪
为了**加速智能体的训练，我们将使用 GPU**。下面的代码将检查 PyTorch 是否可以访问 CUDA enabled 的 GPU。
如果你的计算机上有正确配置的 NVIDIA GPU，你应该会看到 `device:cuda:0`。

In [43]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 创建一个虚拟显示器 🖥

在本地运行时，如果你有桌面环境，通常不需要这一步。但是，如果你在没有显示器的服务器上运行，或者为了确保代码的可移植性，创建一个虚拟屏幕是很有用的。
下面的单元格将安装所需的库并启动一个虚拟屏幕，这样我们就可以在后台渲染环境并录制视频了。


### 在本地安装依赖项
要运行此 Notebook，你需要安装几个 Python 包。在你的终端中运行以下命令来安装它们。

对于视频录制功能，你可能还需要安装 `ffmpeg`。在 Ubuntu/Debian 上，你可以使用 `sudo apt install ffmpeg`。在 macOS 上，可以使用 `brew install ffmpeg`。

```bash
# 安装 Python 包
pip install gym==0.21.0 gym-games==0.2.0 pygame==2.1.0 huggingface-hub==0.8.1 protobuf==3.19.5 imageio==2.9.0 pyvirtualdisplay
```
下面的代码单元格是为了在 Notebook 环境中运行而保留的，但建议在终端中进行安装。

In [ ]:
# %%capture
# !apt install python-opengl # 在某些系统上可能需要
# !apt install ffmpeg
# !apt install xvfb
# !pip install pyvirtualdisplay
# !pip install pyglet==1.5.1

In [ ]:
# 虚拟显示
# 如果你在没有显示器的服务器上运行，或者在本地遇到渲染问题，请取消注释以下代码
try:
    from pyvirtualdisplay import Display
    virtual_display = Display(visible=0, size=(1400, 900))
    virtual_display.start()
except ImportError:
    print("PyVirtualDisplay 未安装，将跳过虚拟显示设置。在本地桌面上通常不需要它。")


## 安装依赖项 🔽
第一步是安装依赖项。我们将安装多个库：

- `gym`
- `gym-games`: 使用 PyGame 制作的额外 gym 环境。
- `huggingface_hub`: 🤗 作为一个中心枢纽，任何人都可以在这里共享和探索模型和数据集。它具有版本控制、指标、可视化等功能，可让你轻松与他人协作。

你可能想知道我们为什么安装 `gym` 而不是 `gymnasium` (gym 的一个更新版本)？**因为我们正在使用的 `gym-games` 尚未更新以支持 `gymnasium`**。

你在这里会遇到的区别：
- 在 `gym` 中，我们没有 `terminated` 和 `truncated`，只有 `done`。
- 在 `gym` 中，使用 `env.step()` 返回 `state, reward, done, info`

你可以在这里了解更多关于 Gym 和 Gymnasium 之间的区别 👉 https://gymnasium.farama.org/content/migration-guide/


你可以在这里看到所有可用的 Reinforce 模型 👉 https://huggingface.co/models?other=reinforce

你可以在这里找到所有深度强化学习模型 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning


In [3]:
# 我们从 GitHub 加载 requirements.txt 以确保版本一致
# 建议在终端中使用 'pip install -r <url>' 或下载文件后 'pip install -r requirements-unit4.txt' 来安装
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit4/requirements-unit4.txt

  Cloning https://github.com/ntasfi/PyGame-Learning-Environment.git to c:\users\10240\appdata\local\temp\pip-req-build-8_9wvlyi
  Resolved https://github.com/ntasfi/PyGame-Learning-Environment.git to commit 3dbe79dc0c35559bb441b9359948aabf9bb3d331
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/simoninithomas/gym-games to c:\users\10240\appdata\local\temp\pip-req-build-m8_yi_s4
  Resolved https://github.com/simoninithomas/gym-games to commit f31695e4ba028400628dc054ee8a436f28193f0b
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached PyYAML-6.0-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0-cp311-cp311-win_amd64.whl (143 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2


  Running command git clone --filter=blob:none --quiet https://github.com/ntasfi/PyGame-Learning-Environment.git 'C:\Users\10240\AppData\Local\Temp\pip-req-build-8_9wvlyi'
  Running command git clone --filter=blob:none --quiet https://github.com/simoninithomas/gym-games 'C:\Users\10240\AppData\Local\Temp\pip-req-build-m8_yi_s4'
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 2.0.1 which is incompatible.
langchain-community 0.3.18 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 2.0.1 which is incompatible.
langchain-core 0.3.40 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
rapidocr-onnxruntime 1.3.24 requires numpy<2.0.0,>=1.19.5, but you have numpy 2.0.1 which is incompatible.


## 导入包 📦
除了导入已安装的库之外，我们还导入：

- `imageio`: 一个帮助我们生成回放视频的库



In [44]:
import numpy as np

from collections import deque

import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym
import gym_pygame

# Hugging Face Hub
from huggingface_hub import notebook_login # 登录我们的 Hugging Face 帐户以上传模型到 Hub
import imageio

我们现在准备好实现我们的 Reinforce 算法了 🔥

# 第一个智能体：玩 CartPole-v1 🤖

## 创建 CartPole 环境并了解其工作原理
### [环境 🎮](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)


### 为什么我们使用像 CartPole-v1 这样的简单环境？
正如[强化学习技巧和窍门](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html)中所解释的，当你从头开始实现你的智能体时，你需要**在进入更深层次之前，确保它在简单的环境中正常工作并找到错误**。因为在简单的环境中找到错误会容易得多。


> 尝试在玩具问题上获得一些“生命迹象”


> 通过在越来越难的环境中运行来验证实现（你可以将结果与 RL zoo 进行比较）。你通常需要为此步骤运行超参数优化。
___
### CartPole-v1 环境

> 一根杆通过一个无驱动的关节连接到一个在无摩擦轨道上移动的推车上。摆锤垂直放置在推车上，目标是通过向推车施加向左和向右的力来平衡杆。



所以，我们从 CartPole-v1 开始。目标是向左或向右推动推车，**以便杆保持平衡。**

如果出现以下情况，回合结束：
- 杆的角度大于 ±12°
- 推车位置大于 ±2.4
- 回合长度大于 500

每当杆保持平衡，我们每个时间步都会获得 +1 的奖励 💰。

In [45]:
env_id = "CartPole-v1"
# 创建环境
env = gym.make(env_id)

# 创建评估环境
eval_env = gym.make(env_id)

# 获取状态空间和动作空间
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [46]:
print("_____观察空间_____ \n")
print("状态空间大小为: ", s_size)
print("观察样本", env.observation_space.sample()) # 获取一个随机观察

_____观察空间_____ 

状态空间大小为:  4
观察样本 [ 1.1961311e+00 -4.3045794e+37  3.1573388e-01 -1.8938621e+38]


In [13]:
print("\n _____动作空间_____ \n")
print("动作空间大小为: ", a_size)
print("动作空间样本", env.action_space.sample()) # 获取一个随机动作


 _____动作空间_____ 

动作空间大小为:  2
动作空间样本 1


## 让我们构建 Reinforce 架构
这个实现基于两个实现：
- [PyTorch 官方强化学习示例](https://github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py)
- [Udacity Reinforce](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb)
- [Chris1nexus 改进的集成](https://github.com/huggingface/deep-rl-class/pull/95)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce"/>

所以我们想要：
- 两个全连接层 (fc1 和 fc2)。
- 使用 ReLU 作为 fc1 的激活函数
- 使用 Softmax 输出一个关于动作的概率分布

In [47]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        # 创建两个全连接层
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        # 定义前向传播
        # 状态进入 fc1 然后我们应用 ReLU 激活函数
        x = F.relu(self.fc1(x))
        # fc1 的输出进入 fc2
        x = self.fc2(x)
        # 我们输出 softmax
        return F.softmax(x, dim=1)

    def act(self, state):
        """
        给定一个状态，采取行动
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = np.argmax(m) # <--- 这里有个错误!
        return action.item(), m.log_prob(action)

我犯了一个错误，你能猜到在哪里吗？

- 为了找出答案，让我们进行一次前向传播：

In [ ]:
debug_policy = Policy(s_size, a_size, 64).to(device)
try:
    debug_policy.act(env.reset())
except Exception as e:
    print(e)

- 这里我们看到错误提示 `ValueError: The value argument to log_prob must be a Tensor`

- 这意味着 `m.log_prob(action)` 中的 `action` 必须是一个张量 **但它不是**。

- 你知道为什么吗？检查 act 函数，试着看看为什么它不起作用。

提示 💡：这个实现中有些东西是错的。记住，`act` 函数**我们想要从动作的概率分布中采样一个动作**。


### (真正的) 解决方案

In [48]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample() # <-- 正确的方法是采样!
        return action.item(), m.log_prob(action)

通过使用 CartPole，调试变得更容易，因为**我们知道错误来自我们的集成，而不是来自我们的简单环境**。

- 因为**我们想从动作的概率分布中采样一个动作**，我们不能使用 `action = np.argmax(m)`，因为它总是会输出概率最高的动作。

- 我们需要用 `action = m.sample()` 来替换，它将从概率分布 P(.|s) 中采样一个动作

### 让我们构建 Reinforce 训练算法
这是 Reinforce 算法的伪代码：

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/pg_pseudocode.png" alt="Policy gradient pseudocode"/>
  

- 当我们计算回报 Gt (伪代码第 6 行) 时，我们看到我们计算的是**从时间步 t 开始**的折扣奖励总和。

- 为什么？因为我们的策略应该只**根据后果来强化动作**：所以在采取一个动作之前获得的奖励是无用的（因为它们不是由该动作引起的），**只有在动作之后发生的奖励才重要**。

- 在编写代码之前，你应该阅读这一部分 [不要让过去分散你的注意力](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#don-t-let-the-past-distract-you)，它解释了我们为什么使用 “未来奖励”(reward-to-go) 策略梯度。

我们使用一种由 [Chris1nexus](https://github.com/Chris1nexus) 编写的有趣技术来**高效地计算每个时间步的回报**。注释解释了该过程。也请随时[查看 PR 的解释](https://github.com/huggingface/deep-rl-class/pull/95)
但总的来说，这个想法是**高效地计算每个时间步的回报**。

你可能会问的第二个问题是**为什么我们要最小化损失**？你说的是梯度上升而不是梯度下降？

- 我们想要最大化我们的效用函数 $J(\theta)$，但在 PyTorch 中，就像在 Tensorflow 中一样，最好是**最小化一个目标函数。**
    - 所以，假设我们想在某个时间步强化动作 3。训练前，这个动作的概率 P 是 0.25。
    - 所以我们想要修改 $\theta$ 使得 $\pi_\theta(a_3|s; \theta) > 0.25$
    - 因为所有的概率 P 之和必须为 1，最大化 $\pi_\theta(a_3|s; \theta)$ 将**最小化其他动作的概率。**
    - 所以我们应该告诉 PyTorch **最小化 $1 - \pi_\theta(a_3|s; \theta)$。**
    - 当 $\pi_\theta(a_3|s; \theta)$ 接近 1 时，这个损失函数趋近于 0。
    - 所以我们是在鼓励梯度去最大化 $\pi_\theta(a_3|s; \theta)$。

在我们的实现中，我们不最小化 $1 - \pi_\theta(a_t|s_t)$，而是最小化 $-\log \pi_\theta(a_t|s_t) * G_t$。这在效果上是相同的：我们想增加一个动作的对数概率，而这个动作会带来高的回报。

In [15]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # 帮助我们在训练期间计算分数
    scores_deque = deque(maxlen=100)
    scores = []
    # 伪代码第 3 行
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset() # TODO: 重置环境
        # 伪代码第 4 行
        for t in range(max_t):
            action, log_prob = policy.act(state) # TODO: 获取动作
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action) # TODO: 在环境中执行一步
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # 伪代码第 6 行：计算回报
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        
        # 从后向前计算每个时间步的折扣回报
        # G_t = r_t + gamma * G_{t+1}
        # G_{T-1} = r_{T-1} (其中 T 是回合的最后一步)
        # 这种方法（动态规划）可以有效地重用计算过的未来回报
        # 避免了 O(N^2) 的朴素计算
        
        # 我们从最后一个时间步开始向第一个时间步计算，
        # 以便利用上面介绍的公式，避免不必要的重复计算。
        # 因此，队列 `returns` 将按时间顺序存储回报，从 t=0 到 t=n_steps
        # 这要归功于 appendleft() 函数，它允许以 O(1) 的常数时间在位置 0 处追加
        # 而普通的 python 列表则需要 O(N) 的时间来完成此操作。
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(rewards[t] + gamma * disc_return_t) # TODO: 在这里完成

        ## 标准化回报可以使训练更稳定
        eps = np.finfo(np.float32).eps.item()

        ## eps 是最小的可表示浮点数，
        # 将其添加到回报的标准差中以避免数值不稳定
        returns = torch.tensor(list(returns))
        returns = (returns - returns.mean()) / (returns.std() + eps)

        # 伪代码第 7 行：计算策略损失
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        # 伪代码第 8 行：PyTorch 偏好梯度下降
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('回合 {}\t平均分数: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores

#### 解决方案

In [49]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # 帮助我们在训练期间计算分数
    scores_deque = deque(maxlen=100)
    scores = []
    # 伪代码第 3 行
    for i_episode in range(1, n_training_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # 伪代码第 4 行
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # 伪代码第 6 行：计算回报
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        
        # 从后向前计算每个时间步的折扣回报
        # G_t = r_t + gamma * G_{t+1}
        # G_{T-1} = r_{T-1} (其中 T 是回合的最后一步)
        # 这种方法（动态规划）可以有效地重用计算过的未来回报
        # 避免了 O(N^2) 的朴素计算
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft(rewards[t] + gamma * disc_return_t)

        ## 标准化回报可以使训练更稳定
        eps = np.finfo(np.float32).eps.item()
        ## eps 是最小的可表示浮点数，
        # 将其添加到回报的标准差中以避免数值不稳定
        returns = torch.tensor(list(returns))
        returns = (returns - returns.mean()) / (returns.std() + eps)

        # 伪代码第 7 行：计算策略损失
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            policy_loss.append(-log_prob * disc_return)
        policy_loss = torch.cat(policy_loss).sum()

        # 伪代码第 8 行：PyTorch 偏好梯度下降
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if i_episode % print_every == 0:
            print('回合 {}\t平均分数: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores

## 训练它
- 我们现在准备好训练我们的智能体了。
- 但首先，我们定义一个包含所有训练超参数的变量。
- 你可以（也应该 😉）更改训练参数

In [ ]:
cartpole_hyperparameters = {
    "h_size": 16, # 隐藏层大小
    "n_training_episodes": 1000, # 训练回合数
    "n_evaluation_episodes": 10, # 评估回合数
    "max_t": 1000, # 每个回合的最大步数
    "gamma": 1.0, # 折扣因子
    "lr": 1e-2, # 学习率
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [ ]:
# 创建策略并将其放置到设备上
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [ ]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"],
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"],
                   100)

## 定义评估方法 📝
- 在这里，我们定义将用于测试我们的 Reinforce 智能体的评估方法。

In [59]:
def evaluate_agent(env, max_steps, n_eval_episodes, policy):
  """
  在 n_eval_episodes 个回合中评估智能体，并返回平均奖励和奖励的标准差。
  :param env: 评估环境
  :param n_eval_episodes: 评估智能体的回合数
  :param policy: Reinforce 智能体
  """
  episode_rewards = []
  for episode in range(n_eval_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action, _ = policy.act(state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward

      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## 评估我们的智能体 📈

In [52]:
evaluate_agent(eval_env,
               cartpole_hyperparameters["max_t"],
               cartpole_hyperparameters["n_evaluation_episodes"],
               cartpole_policy)

NameError: name 'cartpole_hyperparameters' is not defined

### 在 Hub 上发布我们训练好的模型 🔥
现在我们看到训练后取得了不错的结果，我们可以用一行代码将我们训练好的模型发布到 Hub 上 🤗。

这是一个模型卡的示例：

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/modelcard.png"/>

### 推送到 Hub
#### 不要修改此代码

In [53]:
from huggingface_hub import HfApi, snapshot_download
from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import datetime
import json
import imageio

import tempfile

import os

In [54]:
def record_video(env, policy, out_directory, fps=30):
  """
  生成智能体的回放视频
  :param env
  :param policy: 我们智能体的策略
  :param out_directory
  :param fps: 每秒帧数
  """
  images = []
  done = False
  state = env.reset()
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # 根据给定状态采取具有最大预期未来奖励的动作
    action, _ = policy.act(state)
    state, reward, done, info = env.step(action) # 为了录制逻辑，我们直接将 next_state = state
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

In [55]:
def push_to_hub(repo_id,
                model,
                hyperparameters,
                eval_env,
                video_fps=30
                ):
  """
  评估、生成视频并上传模型到 Hugging Face Hub。
  此方法完成整个流程：
  - 评估模型
  - 生成模型卡
  - 生成智能体的回放视频
  - 将所有内容推送到 Hub

  :param repo_id: Hugging Face Hub 上的模型仓库 ID
  :param model: 我们想要保存的 PyTorch 模型
  :param hyperparameters: 训练超参数
  :param eval_env: 评估环境
  :param video_fps: 录制视频回放的每秒帧数
  """

  _, repo_name = repo_id.split("/")
  api = HfApi()

  # 步骤 1: 创建仓库
  repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
  )

  with tempfile.TemporaryDirectory() as tmpdirname:
    local_directory = Path(tmpdirname)

    # 步骤 2: 保存模型
    torch.save(model, local_directory / "model.pt")

    # 步骤 3: 将超参数保存到 JSON
    with open(local_directory / "hyperparameters.json", "w") as outfile:
      json.dump(hyperparameters, outfile)

    # 步骤 4: 评估模型并构建 JSON
    mean_reward, std_reward = evaluate_agent(eval_env,
                                            hyperparameters["max_t"],
                                            hyperparameters["n_evaluation_episodes"],
                                            model)
    # 获取日期时间
    eval_datetime = datetime.datetime.now()
    eval_form_datetime = eval_datetime.isoformat()

    evaluate_data = {
          "env_id": hyperparameters["env_id"],
          "mean_reward": mean_reward,
          "n_evaluation_episodes": hyperparameters["n_evaluation_episodes"],
          "eval_datetime": eval_form_datetime,
    }

    # 写入 JSON 文件
    with open(local_directory / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)

    # 步骤 5: 创建模型卡
    env_name = hyperparameters["env_id"]

    metadata = {}
    metadata["tags"] = [
          env_name,
          "reinforce",
          "reinforcement-learning",
          "custom-implementation",
          "deep-rl-class"
      ]

    # 添加指标
    eval = metadata_eval_result(
        model_pretty_name=repo_name,
        task_pretty_name="reinforcement-learning",
        task_id="reinforcement-learning",
        metrics_pretty_name="mean_reward",
        metrics_id="mean_reward",
        metrics_value=f"{mean_reward:.2f} +/- {std_reward:.2f}",
        dataset_pretty_name=env_name,
        dataset_id=env_name,
      )

    # 合并两个字典
    metadata = {**metadata, **eval}

    model_card = f"""
  # **Reinforce** 智能体玩 **{env_id}**
  这是一个训练好的 **Reinforce** 智能体玩 **{env_id}** 的模型。
  要学习如何使用此模型并训练你自己的模型，请查看深度强化学习课程的第 4 单元：https://huggingface.co/deep-rl-course/unit4/introduction
  """

    readme_path = local_directory / "README.md"
    readme = ""
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
          readme = f.read()
    else:
      readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
      f.write(readme)

    # 将我们的指标保存到 Readme 元数据中
    metadata_save(readme_path, metadata)

    # 步骤 6: 录制视频
    video_path =  local_directory / "replay.mp4"
    record_video(eval_env, model, video_path, video_fps)

    # 步骤 7: 将所有内容推送到 Hub
    api.upload_folder(
          repo_id=repo_id,
          folder_path=local_directory,
          path_in_repo=".",
    )

    print(f"你的模型已推送到 Hub。你可以在这里查看你的模型：{repo_url}")

通过使用 `push_to_hub`，你**评估、录制回放、生成智能体的模型卡并将其推送到 Hub**。

这样：
- 你可以**展示我们的工作** 🔥
- 你可以**可视化你的智能体玩游戏** 👀
- 你可以**与社区共享一个其他人可以使用的智能体** 💾
- 你可以**访问一个排行榜 🏆 来看看你的智能体与同学相比表现如何** 👉 https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard


为了能够与社区共享你的模型，还需要遵循三个步骤：

1️⃣（如果尚未完成）在 HF 上创建一个帐户 ➡ https://huggingface.co/join

2️⃣ 登录，然后，你需要从 Hugging Face 网站存储你的身份验证令牌。
- 创建一个新令牌 (https://huggingface.co/settings/tokens) **并赋予写入角色**


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">


In [56]:
notebook_login()

如果你不想使用 Jupyter Notebook，你需要改用这个命令：`huggingface-cli login`

3️⃣ 我们现在准备好使用 `push_to_hub()` 函数将我们训练好的智能体推送到 🤗 Hub 🔥

In [ ]:
# TODO: 定义你的 repo_id {你的用户名/Reinforce-CartPole-v1}
# 例如：repo_id = "ThomasSimonini/Reinforce-CartPole-v1"
repo_id = "" 
push_to_hub(repo_id,
                cartpole_policy, # 我们想要保存的模型
                cartpole_hyperparameters, # 超参数
                eval_env, # 评估环境
                video_fps=30
                )

现在我们已经测试了我们实现的鲁棒性，让我们尝试一个更复杂的环境：PixelCopter 🚁




## 第二个智能体：PixelCopter 🚁

### 研究 PixelCopter 环境 👀
- [环境文档](https://pygame-learning-environment.readthedocs.io/en/latest/user/games/pixelcopter.html)


In [39]:
env_id = "Pixelcopter-PLE-v0"
env = gym.make(env_id)
eval_env = gym.make(env_id)
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [20]:
print("_____观察空间_____ \n")
print("状态空间大小为: ", s_size)
print("观察样本", env.observation_space.sample()) # 获取一个随机观察

_____观察空间_____ 

状态空间大小为:  7
观察样本 [-0.04964773 -1.9876798  -0.75425804  1.433291   -0.6352314  -0.887026
  0.86039406]


In [21]:
print("\n _____动作空间_____ \n")
print("动作空间大小为: ", a_size)
print("动作空间样本", env.action_space.sample()) # 获取一个随机动作


 _____动作空间_____ 

动作空间大小为:  2
动作空间样本 1


观察空间 (7) 👀:
- 玩家 y 坐标
- 玩家速度
- 玩家到地面的距离
- 玩家到天花板的距离
- 下一个障碍物与玩家的 x 距离
- 下一个障碍物顶部 y 坐标
- 下一个障碍物底部 y 坐标

动作空间(2) 🎮:
- 向上 (按下加速器)
- 什么都不做 (不按加速器)

奖励函数 💰:
- 每通过一个垂直障碍物，它会获得 +1 的正奖励。每次达到终止状态，它会收到 -1 的负奖励。

### 定义新的策略 🧠
- 由于环境更复杂，我们需要一个更深层的神经网络

In [22]:
# 在这里定义一个新的策略类
# 与 CartPole 不同，PixelCopter 更复杂
# 我们将使用一个具有三层的更深层网络
class PixelCopterPolicy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(PixelCopterPolicy, self).__init__()
        # 在这里定义三层
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h_size*2)
        self.fc3 = nn.Linear(h_size*2, a_size)

    def forward(self, x):
        # 在这里定义前向传播过程
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

### 定义超参数 ⚙️
- 因为这个环境更复杂。
- 我们需要更多的神经元，特别是对于隐藏层。

In [23]:
pixelcopter_hyperparameters = {
    "h_size": 64,
    "n_training_episodes": 50000,
    "n_evaluation_episodes": 10,
    "max_t": 10000,
    "gamma": 0.99,
    "lr": 1e-4,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

### 训练它
- 我们现在准备好训练我们的智能体了 🔥。

In [26]:
# 创建策略并将其放置到设备上
# torch.manual_seed(50)
pixelcopter_policy = PixelCopterPolicy(pixelcopter_hyperparameters["state_space"],pixelcopter_hyperparameters["action_space"], pixelcopter_hyperparameters["h_size"]).to(device)
pixelcopter_optimizer = optim.Adam(pixelcopter_policy.parameters(), lr=pixelcopter_hyperparameters["lr"])

In [28]:
scores = reinforce(
    pixelcopter_policy,
    pixelcopter_optimizer,
    pixelcopter_hyperparameters["n_training_episodes"],
    pixelcopter_hyperparameters["max_t"],
    pixelcopter_hyperparameters["gamma"],
    1000,
)

回合 1000	平均分数: 4.45
回合 2000	平均分数: 6.49
回合 3000	平均分数: 9.65
回合 4000	平均分数: 9.38
回合 5000	平均分数: 10.92
回合 6000	平均分数: 13.16
回合 7000	平均分数: 16.60
回合 8000	平均分数: 20.44
回合 9000	平均分数: 21.16
回合 10000	平均分数: 21.29
回合 11000	平均分数: 23.74
回合 12000	平均分数: 25.15
回合 13000	平均分数: 20.83
回合 14000	平均分数: 26.05
回合 15000	平均分数: 21.60
回合 16000	平均分数: 30.59
回合 17000	平均分数: 27.95
回合 18000	平均分数: 27.53
回合 19000	平均分数: 31.01
回合 20000	平均分数: 24.79
回合 21000	平均分数: 33.51
回合 22000	平均分数: 34.43
回合 23000	平均分数: 29.17
回合 24000	平均分数: 33.71
回合 25000	平均分数: 37.35
回合 26000	平均分数: 29.06
回合 27000	平均分数: 21.70
回合 28000	平均分数: 25.08
回合 29000	平均分数: 29.40
回合 30000	平均分数: 36.96
回合 31000	平均分数: 43.21
回合 32000	平均分数: 30.58
回合 33000	平均分数: 43.62
回合 34000	平均分数: 46.33
回合 35000	平均分数: 53.47
回合 36000	平均分数: 37.60
回合 37000	平均分数: 23.46
回合 38000	平均分数: 52.37
回合 39000	平均分数: 54.17
回合 40000	平均分数: 44.95
回合 41000	平均分数: 53.32
回合 42000	平均分数: 46.35
回合 43000	平均分数: 37.40
回合 44000	平均分数: 56.35
回合 45000	平均分数: 58.51
回合 46000	平均分数: 49.61
回合 47000	平均分数: 66.16
回合 48000	平均分数: 67.40
回合 49

In [63]:
import os
import torch

# 保存模型到当前工作目录
torch.save(pixelcopter_policy, "pixelcopter_policy.pt")


### 在 Hub 上发布我们训练好的模型 🔥

In [61]:
# TODO: 定义你的 repo_id {你的用户名/Reinforce-Pixelcopter-v0}
repo_id = "a1024053774/Reinforce-Pixelcopter-v0"
push_to_hub(
    repo_id,
    pixelcopter_policy,  # The model we want to save
    pixelcopter_hyperparameters,  # Hyperparameters
    eval_env,  # Evaluation environment
    video_fps=30
)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 7x64)

## 一些额外的挑战 🏆
最好的学习方式是**亲自动手尝试**！正如你所看到的，当前的智能体表现并不出色。作为第一个建议，你可以训练更多的步数。但也可以尝试寻找更好的参数。

在[排行榜](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard)上，你会找到你的智能体。你能登上榜首吗？

这里有一些实现这一目标的方法：
* 训练更多步数
* 通过查看你同学的做法来尝试不同的超参数 👉 https://huggingface.co/models?other=reinforce
* **将你新训练的模型推送到 Hub** 🔥
* **为更复杂的环境改进实现**（例如，将网络更改为卷积神经网络以处理
作为观察的帧怎么样）？

________________________________________________________________________

**恭喜你完成本单元**！这里有很多信息。
也恭喜你完成了本教程。你刚刚使用 PyTorch 从头开始编写了你的第一个深度强化学习智能体，并将其分享到了 Hub 上 🥳。

不要犹豫，通过**为更复杂的环境改进实现来迭代本单元**（例如，将网络更改为卷积神经网络以处理作为观察的帧怎么样）？

在下一个单元中，**我们将通过在 Unity 环境中训练智能体来学习更多关于 Unity MLAgents 的知识**。这样，你将准备好参加 **AI vs AI 挑战赛，在那里你将训练你的智能体
在雪球大战和足球比赛中与其他智能体竞争。**

听起来很有趣？下次见！

最后，我们很想**听听你对课程的看法以及我们如何改进它**。如果你有任何反馈，请 👉  [填写此表格](https://forms.gle/BzKXWzLAGZESGNaE9)

我们在第 5 单元见！🔥

### 继续学习，保持出色 🤗

